<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
# Crear el mapa centrado en una ubicación inicial
launch_map = folium.Map(location=[28.563, -80.577], zoom_start=5)

# Datos ficticios de sitios de lanzamiento (reemplázalos con los tuyos)
launch_sites = pd.DataFrame({
    'Name': ['CCAFS LC-40', 'CCAFS SLC-41', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitude': [28.5623, 28.5619, 28.5733, 34.6328],
    'Longitude': [-80.5774, -80.5772, -80.6081, -120.6108]
})

# Agregar marcadores en el mapa
for index, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Mostrar el mapa
launch_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Inicializar el mapa centrado en NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Agregar un círculo y un marcador para cada sitio de lanzamiento
for index, row in launch_sites_df.iterrows():
    # Crear un círculo en la ubicación del sitio de lanzamiento
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Radio del círculo en metros
        color='#000000',  # Color del borde del círculo
        fill=True,
        fill_color='#000000'  # Color de relleno del círculo
    ).add_child(folium.Popup(row['Launch Site']))  # Agregar el nombre como etiqueta emergente

    # Crear un marcador con una etiqueta de texto
    marker = folium.map.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )

    # Agregar el círculo y el marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostrar el mapa con los sitios de lanzamiento
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
# Crear un mapa centrado en NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Definir colores para éxito y fracaso
colors = {1: 'green', 0: 'red'}

# Agregar marcadores para cada lanzamiento
for index, row in spacex_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,  # Tamaño pequeño para diferenciar lanzamientos
        color=colors[row['class']],  # Verde para éxito, rojo para fallo
        fill=True,
        fill_color=colors[row['class']],
        fill_opacity=0.7,
        popup=f"Launch Site: {row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(site_map)

# Mostrar el mapa con los lanzamientos
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Crear una nueva columna 'marker_color' en función de la columna 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Mostrar los primeros registros para verificar
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Agregar el objeto MarkerCluster al mapa
marker_cluster = MarkerCluster().add_to(site_map)

# Recorrer cada fila del DataFrame spacex_df
for index, record in spacex_df.iterrows():
    # Crear un marcador con color basado en el éxito o fallo del lanzamiento
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa')  # Icono de cohete
    )
    
    # Agregar el marcador al MarkerCluster
    marker_cluster.add_child(marker)

# Mostrar el mapa con los lanzamientos en clústeres
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, cos, sin, asin, sqrt

# Función para calcular la distancia entre dos coordenadas usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias entre las coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la Tierra en kilómetros
    R = 6371  
    return c * R  # Distancia en km

# Definir una ubicación de referencia (ejemplo: una carretera cerca de CCAFS SLC-40)
road_coordinate = (28.5623, -80.5707)  # Latitud y longitud aproximadas de una carretera

# Calcular la distancia desde el sitio de lanzamiento CCAFS SLC-40 a la carretera
launch_site = (28.563197, -80.576820)  # Coordenadas de CCAFS SLC-40
distance_km = haversine(launch_site[0], launch_site[1], road_coordinate[0], road_coordinate[1])

# Mostrar la distancia calculada
print(f"Distancia entre CCAFS SLC-40 y la carretera más cercana: {distance_km:.2f} km")


Distancia entre CCAFS SLC-40 y la carretera más cercana: 0.61 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
from math import sin, cos, sqrt, atan2, radians

# Función para calcular la distancia entre dos puntos
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5721  # Ejemplo de coordenadas
launch_site_lon = -80.648  # Ejemplo de coordenadas

# Coordenadas de la costa más cercana
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcular la distancia
distance_coastline = calculate_distance(
    launch_site_lat, 
    launch_site_lon, 
    coastline_lat, 
    coastline_lon
)

print(f"La distancia al punto más cercano de la costa es {distance_coastline:.2f} km")


La distancia al punto más cercano de la costa es 7.52 km


In [24]:
import folium
from folium.features import DivIcon

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Coordenadas de la costa más cercana
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcular la distancia
distance_coastline = calculate_distance(
    launch_site_lat, 
    launch_site_lon, 
    coastline_lat, 
    coastline_lon
)

# Crear el mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Añadir un marcador para el sitio de lanzamiento
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Sitio de lanzamiento",
    icon=folium.Icon(color="blue", icon="rocket")
)
site_map.add_child(launch_site_marker)

# Crear y añadir el marcador para el punto de la costa
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# Mostrar el mapa
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# Create a folium.PolyLine object using the coastline coordinates and launch site coordinate
launch_site_lat = 28.5721  # Coordenadas del sitio de lanzamiento
launch_site_lon = -80.648  # Coordenadas del sitio de lanzamiento

coastline_lat = 28.56367  # Coordenadas de la costa más cercana
coastline_lon = -80.57163  # Coordenadas de la costa más cercana

# Crear una lista de coordenadas para la línea
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Crear el objeto folium.PolyLine
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# Añadir la línea al mapa
site_map.add_child(lines)

# Mostrar el mapa
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [27]:
import folium
from folium.features import DivIcon

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Coordenadas del punto más cercano (por ejemplo, una ciudad, una vía de tren, etc.)
# Cambia estas coordenadas al punto que deseas marcar
closest_point_lat = 28.6024  # Coordenadas de ejemplo de una ciudad cercana
closest_point_lon = -80.6782  # Coordenadas de ejemplo de una ciudad cercana

# Calcular la distancia entre el sitio de lanzamiento y el punto más cercano
distance_to_closest_point = calculate_distance(
    launch_site_lat, 
    launch_site_lon, 
    closest_point_lat, 
    closest_point_lon
)

# Crear un mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Añadir un marcador para el sitio de lanzamiento
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Sitio de lanzamiento",
    icon=folium.Icon(color="blue", icon="rocket")
)
site_map.add_child(launch_site_marker)

# Crear y añadir un marcador para el punto más cercano
closest_point_marker = folium.Marker(
    [closest_point_lat, closest_point_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_closest_point),
    )
)
site_map.add_child(closest_point_marker)

# Crear una línea que conecte el sitio de lanzamiento con el punto más cercano
line_coordinates = [[launch_site_lat, launch_site_lon], [closest_point_lat, closest_point_lon]]
line = folium.PolyLine(locations=line_coordinates, weight=1, color='green')
site_map.add_child(line)

# Mostrar el mapa
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
